In [1]:
#import libraries
import requests
import re
from bs4 import BeautifulSoup
from requests import get
url = "https://hackr.io/tutorials/learn-prototyping"
response = get(url)
#We’ll now create a BeautifulSoup object, or a parse tree. This object takes as its arguments the page.text document from Requests (the content of the server’s response) and then parses it from Python’s built-in html.parser.
soup = BeautifulSoup(response.text, 'html.parser')

# Pull all text from the tuturial divison that contains course listing
courses = soup.find_all('div', class_ ="tut-list tut-list-new tut-row")
print(type(courses))
print(len(courses))

<class 'bs4.element.ResultSet'>
3


Now we’ll select each container, and extract, by turn, each item of interest:

- Course title
- Votes
- Author
- Links (Course, Author, Hackr)
- Course Features: (Free / Paid) (video / book ) (Beginner / Intermediate / Advanced)


In [2]:
# Lists to store the scraped data in
CourseTitles = []
Votes = []
CourseLink = []
AuthorName = []
AuthorLink = []
HackrLink = []
CourseCost = []
CourseType = []
StudentType = []


    
for course in courses:
    CourseTitle = course.div.find('span', class_ = 'tutorial-title-txt').text.strip()
    CourseTitles.append(CourseTitle)


    NoOfVote  = course.div.find('span', class_ = 'count').text.strip()
    Votes.append(NoOfVote)

    Link1 = course.find("div", class_="title-links no-margin").find('a')["href"]
    CourseLink.append(Link1)
    
    Name = course.find("a", class_="author no-marginright paddingleft-xs")
    if Name is None:
        AuthorName.append(None)
    else:
        Names=Name.text.strip()
        AuthorName.append(Names)
    
    auth  = course.find("div", class_="action author").find('a')
    if auth is None:
        AuthorLink.append(None)
    else:
        Link2 = auth["href"]
        AuthorLink.append(Link2)
    
    hackr = course.find("a", attrs={'href': re.compile("https://hackr.io/tutorial/")}) 
    if hackr is None:
        HackrLink.append(None)
    else:
        Link3=hackr.get("href")
        HackrLink.append(Link3)
        
    features  = course.find_all('div', class_ = "tut-label-group marginright-sm")
    for future in features:
        x=future.text.split()
    feat1 = x[0]
    if len(x) > 2:
        feat2 = x[1]
        feat3 = x[2]
    elif x[1] == ("Beginner" or "Advanced"):
        feat2 =None
        feat3= x[1]
    else:
        feat2 = x[1]
        feat3 = None
    CourseCost.append(feat1)
    CourseType.append(feat2)
    StudentType.append(feat3)
        

print (f"Courses : {CourseTitles}\n")
print (f"Votes : {Votes}\n")
print (f"CourseLink : {CourseLink}\n")
print (f"Author : {AuthorName}\n")
print (f"AuthorLink : {AuthorLink}\n")
print (f"HackrLink : {HackrLink}\n")
print (f"CourseLink : {CourseLink}\n")
print (f"CourseCost : {CourseCost}\n")
print (f"CourseType : {CourseType}\n")
print (f"StudentType : {StudentType}\n")


Courses : ['UX Foundations: Prototyping', 'Fast Prototyping with Sketch, Invision and Craft', 'Rapid Prototyping']

Votes : ['1', '2', '1']

CourseLink : ['https://linkedin-learning.pxf.io/MMDXn', 'https://www.youtube.com/watch?v=5lg-PbDZEn8', 'https://www.anrdoezrs.net/links/8752481/type/dlg/https://www.udacity.com/course/rapid-prototyping--ud723/']

Author : ['Nick', None, 'André']

AuthorLink : ['https://hackr.io/nick-feng', None, 'https://hackr.io/franciosi']

HackrLink : ['https://hackr.io/tutorial/ux-foundations-prototyping', None, 'https://hackr.io/tutorial/rapid-prototyping']

CourseLink : ['https://linkedin-learning.pxf.io/MMDXn', 'https://www.youtube.com/watch?v=5lg-PbDZEn8', 'https://www.anrdoezrs.net/links/8752481/type/dlg/https://www.udacity.com/course/rapid-prototyping--ud723/']

CourseCost : ['Video', 'Free', 'Free']

CourseType : ['Beginner', 'Video', 'Video']

StudentType : ['Paid', 'Beginner', None]



In [3]:
import pandas as pd
PrototypingCourses = pd.DataFrame({'Course': CourseTitles,
'Votes': Votes,
"Course Link":CourseLink,
"Author ":AuthorName,
"Author Link":AuthorLink,
"Hackr Link":HackrLink,
"Course Cost":CourseCost,
"Course Type":CourseType,
"Course Cost":CourseCost,
"Student Type":StudentType
})
print(PrototypingCourses.info())
PrototypingCourses

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Course        3 non-null      object
 1   Votes         3 non-null      object
 2   Course Link   3 non-null      object
 3   Author        2 non-null      object
 4   Author Link   2 non-null      object
 5   Hackr Link    2 non-null      object
 6   Course Cost   3 non-null      object
 7   Course Type   3 non-null      object
 8   Student Type  2 non-null      object
dtypes: object(9)
memory usage: 344.0+ bytes
None


,Course,Votes,Course Link,Author,Author Link,Hackr Link,Course Cost,Course Type,Student Type
0,UX Foundations: Prototyping,1,https://linkedin-learning.pxf.io/MMDXn,Nick,https://hackr.io/nick-feng,https://hackr.io/tutorial/ux-foundations-proto...,Video,Beginner,Paid
1,"Fast Prototyping with Sketch, Invision and Craft",2,https://www.youtube.com/watch?v=5lg-PbDZEn8,None,None,None,Free,Video,Beginner
2,Rapid Prototyping,1,https://www.anrdoezrs.net/links/8752481/type/d...,André,https://hackr.io/franciosi,https://hackr.io/tutorial/rapid-prototyping,Free,Video,None


In [4]:
PrototypingCourses.to_csv('PrototypingCourses.csv')